<a href="https://colab.research.google.com/github/VedantDesai11/3c-net_on_mpcooking2/blob/main/3c_net_mpcooking2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement 3C-Net: Category Count and Center Loss for Weakly-Supervised Action Localization paper on mpcooking2 dataset.



In [ ]:
!git clone https://github.com/naraysa/3c-net 
!pip install tensorboard_logger

# Change model.py

```
#base_x_f = inputs[:,:,1024:]
base_x_f = inputs[:,:,128:]
#base_x_r = inputs[:,:,:1024]
base_x_r = inputs[:,:,:128]
```

# Upload read_cooking_dataset.py Class file

In [ ]:
# TESTING CELL 


import numpy as np
'''

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
a = np.load('/content/3c-net/Thumos14-Annotations/classlist.npy')
b = np.load('/content/3c-net/Thumos14-Annotations/labels.npy')

# restore np.load for future normal usage
np.load = np_load_old

print(a)


#import pickle

#reza_dataset = pickle.load(open('/content/drive/My Drive/MPCooking 2/mpcooking2_data_pca.pickle', "rb"))

print reza_dataset.train[0][0] # video name
#print reza_dataset.train[0][1].shape # action label per frame
#print np.array([item[1] for item in reza_dataset.train]).shape
#print len(reza_dataset.train)
#print reza_dataset.train[0][2] # actions order label
#print(reza_dataset.train[0][3].shape)# rgb features
#print(reza_dataset.train[0][4].shape) # flow features
print(len(reza_dataset.dish_classlist)) # video name
print(utils.strlist2multihot([reza_dataset.dish_classlist[0]],reza_dataset.dish_classlist))
'''

NameError: ignored

In [ ]:
# Cell to import dataset for checking errors in model

import pickle

print('loading pickle')
reza_dataset = pickle.load(open('/content/drive/My Drive/MPCooking 2/mpcooking2_data_pca.pickle', "rb"))
print('pickle loaded')

In [ ]:
import pickle
import os
import numpy as np
import utils

class ConvertDataset:
     def __init__(self, reza_dataset, args):
         self.dataset_name = args.dataset_name
         self.feature_size = 256
         self.num_class = reza_dataset.nDishes # number of class = number of dishes in our case (59dishes)
         self.classlist = reza_dataset.dish_classlist
         self.t_max = None
         self.is_training = False
         self.batch_size = args.batch_size
         self.train = reza_dataset.train
         self.test = reza_dataset.test
         self.currenttestidx = 0
         self.testidx = [i for i in range(len(reza_dataset.test))]
         #path_to_annotations
         #lst_valid
         self.labels101to20 = np.arange(0,59)
         self.currenttrainidx = 0
        
     def load_data(self, is_training=True):

        features = []
        labels = []
        count_labels = []
        if is_training:

            # Not performing random sampling here, going through training data in order
            if self.currenttrainidx + self.batch_size < len(self.train):
                batch_end = self.batch_size
            else:
                batch_end = len(self.train)

            for i in range(self.currenttrainidx, self.currenttrainidx + batch_end):

                # get label for recipe from video name ex: s02-d72 72 is dish index. get dish name from dish_index2label(dish index)
                label = self.train[i][0].split('-')[1].replace('d','')
                dish_name = reza_dataset.dish_index2label[label]
                labels.append([dish_name])

                # concat the rgb and flow features
                flow = self.train[i][4] # flow features
                rgb = self.train[i][3] # rgb features
                features.append(np.concatenate((flow, rgb)).T) #transpose features to get (length x num_features)

                # our recipe is the action so count for each is 1
                count_labels.append(1)
            
            labels_multihot = [utils.strlist2multihot(labs,self.classlist) for labs in labels]
            
            return np.array([utils.process_feat(feature, self.t_max) for feature in features]), np.array(labels_multihot), np.array(count_labels)

        else:
            testing_data = self.test[self.currenttestidx]

            # getting the label for that testing data
            string_label = reza_dataset.dish_index2label[testing_data[0].split('-')[1].replace('d','')]
            labs = utils.strlist2multihot([string_label],self.classlist)

            # getting the features for that testing data
            flow = testing_data[4] # flow features
            rgb = testing_data[3] # rgb features
            feat = np.concatenate((flow, rgb)).T

            if self.currenttestidx == len(self.testidx)-1:
                done = True; self.currenttestidx = 0
            else:
                done = False; self.currenttestidx += 1
         
            return np.array([feat]), np.array(labs), done
            

In [ ]:
from __future__ import print_function
import argparse
import os
import torch
from model import Model
from video_dataset import Dataset
from test import test
from train import train
from tensorboard_logger import Logger
import options
from center_loss import CenterLoss
torch.set_default_tensor_type('torch.cuda.FloatTensor')
import torch.optim as optim

import pickle

if __name__ == '__main__':

    args = options.parser.parse_args(['--dataset-name', 'MP_Cooking_2'])
    
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    device = torch.device("cuda")
    
    # Change T_MAX --
    t_max = 750
    t_max_ctc = 7500
    if args.activity_net:
        t_max = 200
        t_max_ctc = 400


    #dataset = Dataset(args)
    dataset = ConvertDataset(reza_dataset, args)
    
    #Change save directory to drive to avoid loss of data
    os.system('mkdir -p ./drive/My Drive/MPCooking 2/ckpt/')
    os.system('mkdir -p ./drive/My Drive/MPCooking 2/logs/' + args.model_name)
    logger = Logger('./drive/My Drive/MPCooking 2/logs/' + args.model_name)
    
    #model = Model(dataset.feature_size, dataset.num_class, dataset.labels101to20).to(device)
    model = Model(dataset.feature_size, dataset.num_class).to(device)

    if args.eval_only and args.pretrained_ckpt is None:
        print('***************************')
        print('Pretrained Model NOT Loaded')
        print('Evaluating on Random Model')
        print('***************************')

    if args.pretrained_ckpt is not None:
        model.load_state_dict(torch.load(args.pretrained_ckpt))

    best_acc = 0
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.0005)  
    #criterion_cent_f = CenterLoss(num_classes=dataset.num_class, feat_dim=1024, use_gpu=True)
    criterion_cent_f = CenterLoss(num_classes=dataset.num_class, feat_dim=128, use_gpu=True)
    optimizer_centloss_f = torch.optim.SGD(criterion_cent_f.parameters(), lr=0.1)
    #criterion_cent_r = CenterLoss(num_classes=dataset.num_class, feat_dim=1024, use_gpu=True)
    criterion_cent_r = CenterLoss(num_classes=dataset.num_class, feat_dim=128, use_gpu=True)
    optimizer_centloss_r = torch.optim.SGD(criterion_cent_r.parameters(), lr=0.1)

    criterion_cent_all=[criterion_cent_f, criterion_cent_r]
    optimizer_centloss_all=[optimizer_centloss_f, optimizer_centloss_r]

    for itr in range(args.max_iter):
        dataset.t_max = t_max
        if itr % 2 == 0 and itr > 000:
            dataset.t_max = t_max_ctc
        if not args.eval_only:
            train(itr, dataset, args, model, optimizer, criterion_cent_all, optimizer_centloss_all, logger, device)
          
        if itr % 500 == 0 and (not itr == 0 or args.eval_only):
            acc = test(itr, dataset, args, model, logger, device)
            print(args.summary)
            if acc > best_acc and not args.eval_only:
              torch.save(model.state_dict(), './drive/My Drive/MPCooking 2/ckpt/' + args.model_name + '.pkl')
              best_acc = acc
        if args.eval_only:
            print('Done Eval!')
            break


Iteration: 0, Loss: 12.341
Iteration: 1, Loss: 12.466
Iteration: 2, Loss: 47.814
Iteration: 3, Loss: 12.526
Iteration: 4, Loss: 47.003
Iteration: 5, Loss: 12.420
Iteration: 6, Loss: 46.068
Iteration: 7, Loss: 12.493
Iteration: 8, Loss: 45.321
Iteration: 9, Loss: 12.459
Iteration: 10, Loss: 44.563
Iteration: 11, Loss: 12.341
Iteration: 12, Loss: 43.535
Iteration: 13, Loss: 12.367
Iteration: 14, Loss: 42.755
Iteration: 15, Loss: 12.373
Iteration: 16, Loss: 41.712
Iteration: 17, Loss: 12.396
Iteration: 18, Loss: 40.798
Iteration: 19, Loss: 12.318
Iteration: 20, Loss: 40.169
Iteration: 21, Loss: 12.299
Iteration: 22, Loss: 39.035
Iteration: 23, Loss: 12.307
Iteration: 24, Loss: 38.219
Iteration: 25, Loss: 12.416
Iteration: 26, Loss: 37.411
Iteration: 27, Loss: 12.343
Iteration: 28, Loss: 36.471
Iteration: 29, Loss: 12.293
Iteration: 30, Loss: 35.763
Iteration: 31, Loss: 12.316
Iteration: 32, Loss: 34.902
Iteration: 33, Loss: 12.249
Iteration: 34, Loss: 34.186
Iteration: 35, Loss: 12.295
It

classificationMAP.py:10: RuntimeWarning: invalid value encountered in divide
  rec=tp/npos; prec=tp/(fp+tp)
classificationMAP.py:13: RuntimeWarning: invalid value encountered in divide
  return np.sum(tmp*prec)/npos


nan


AttributeError: ignored